In [124]:
from collections import deque
# from aiops_detection_online.detection_service.common import mongodb_helper, kafka_helper
from sklearn.externals import joblib
import pymysql
import fbprophet
# from aiops_detection_online.config import server_config
import os
import time
import pandas as pd
mysql_dic={'mysql': {
        'server': '58.16.78.136',
        'port': 23306,
        'user': 'root',
        'passwd': '1234',
        'db': 'aiops_monitor'
    }}
host = mysql_dic.get("server")
port = mysql_dic.get("port")
db_name = mysql_dic.get("db")
user = mysql_dic.get('user')
passwd = mysql_dic.get('passwd')

d = pymysql.connect(db='aiops_monitor', user='root', passwd='1234', host='58.16.78.136', port=23306)
cursor = d.cursor()

# time_local = time.localtime(timestamp)
# 转换成新的时间格式(2016-05-05 20:28:54)
# dt = time.strftime("%Y-%m-%d %H:%M:%S", time_local)
v = '840'

# sql = "SELECT * FROM predict where item_id=%s"%v
sql="select predict_time,value from aiops_monitor.predict where item_id='%d' order by predict_time desc limit 100" % (840) 
# 执行SQL语句

# 读取sql
data_sql=pd.read_sql(sql,d)
# 存储
# data_sql.to_csv("test.csv")
# cursor.execute(sql, v)
# results = cursor.fetchall()
d.close()
data_sql

,predict_time,value
0,2019-05-29 14:54:50,34577.97
1,2019-05-29 14:54:50,34577.97
2,2019-05-29 14:54:50,34577.97
3,2019-05-29 14:54:50,34577.97
4,2019-05-29 14:54:50,34577.97
5,2019-05-29 14:54:50,34577.97
6,2019-05-29 14:54:50,34577.97
7,2019-05-29 14:54:50,34577.97
8,2019-05-29 14:54:50,34577.97
9,2019-05-29 14:54:50,34577.97


In [138]:
import time

dt = str(data_sql.loc[99,:][0])

#转换成时间数组
timeArray = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
#转换成时间戳
timestamp =int(time.mktime(timeArray))

timestamp

1559112890

In [ ]:
'kafka': {
        'server_1': {
            'host': '10.1.11.16',
            'port': 9092,
        },
        'server_2': {
            'host': '10.1.11.14',
            'port': 9092,
        },
        'group': 'detection_online',
        'std_topic': 'std-item',
        'alarm_topic': 'alarm-data',
        'auto_offset_reset': 'latest'  # latest or earliest
    }

In [ ]:

from kafka import KafkaConsumer

consumer = KafkaConsumer('model',auto_offset_reset='earliest',bootstrap_servers='58.16.78.136:64547')

for msg in consumer:
    print(eval(msg.value))

In [142]:
import datetime
import pandas as pd
import pymongo
train_history = collections.deque()

myclient = pymongo.MongoClient('58.16.78.136:27017')
mydb = myclient["test"]
mycol = mydb["item"]

import time
import datetime


data_temp=[]
myquery = {"itemId": 840,"recordTime":{ "$gt": 1559112890, "$lt": 1559115890 } }
for x in mycol.find(myquery, {"recordTime": 1, "value": 1}):
    td = x
    data_temp.append([td.get('recordTime'), td.get('value')])

[[1559112926, '2961'],
 [1559112986, '2910'],
 [1559113046, '2931'],
 [1559112891, '3304.0'],
 [1559112892, '2594.0'],
 [1559112893, '2675.0'],
 [1559112894, '3102.0'],
 [1559112895, '3055.0'],
 [1559112896, '2935.0'],
 [1559112897, '2980.0'],
 [1559112900, '2856.0'],
 [1559112899, '2876.0'],
 [1559112902, '2943.0'],
 [1559112898, '2636.0'],
 [1559112901, '2894.0'],
 [1559112903, '3033.0'],
 [1559112905, '2840.0'],
 [1559112904, '2965.0'],
 [1559112907, '3009.0'],
 [1559112908, '2776.0'],
 [1559112910, '2881.0'],
 [1559112906, '2872.0'],
 [1559112911, '2758.0'],
 [1559112909, '2781.0'],
 [1559112912, '3015.0'],
 [1559112913, '2885.0'],
 [1559112914, '2700.0'],
 [1559112916, '2729.0'],
 [1559112915, '3582.0'],
 [1559112917, '2652.0'],
 [1559112918, '2892.0'],
 [1559112919, '2707.0'],
 [1559112920, '2796.0'],
 [1559112922, '3543.0'],
 [1559112921, '3099.0'],
 [1559112924, '3009.0'],
 [1559112923, '2872.0'],
 [1559112925, '3089.0'],
 [1559112927, '2728.0'],
 [1559112928, '2617.0'],
 [1559

In [143]:
len(data_temp)

3059

In [83]:
d.columns=['ds','y']

In [84]:
d[d['ds']==1559154748]

,ds,y
55673,1559154748,2636.0


In [86]:
pd.to_datetime(1559154748,unit='s',)

Timestamp('2019-05-29 18:32:28')

In [66]:
d9531=d953.tail(300)

In [74]:
d=d.reset_index()
d9531=d9531.reset_index()

In [76]:
d9531.drop(['index','predict_time','id','ds'],axis=1,inplace=True)

In [78]:
d9531['predict_time']=d['ds']

In [123]:
import pymysql
 
conn = pymysql.connect(db='aiops_monitor', user='root', passwd='1234', host='58.16.78.136', port=23306)
cursor = conn.cursor()
 

# sql="select user,pass from tb7 where user='%s' and pass='%s'" % (user,passwd)
sql="select predict_time,value from aiops_monitor.predict where item_id='%d' order by predict_time desc limit 100" % (840) 
#拼接语句被构造成下面这样，永真条件，此时就注入成功了。因此要避免这种情况需使用pymysql提供的参数化查询。
#select user,pass from tb7 where user='u1' or '1'-- ' and pass='u1pass'
 
cursor.execute(sql)

print(cursor.fetchall())
 
conn.commit()
cursor.close()
conn.close()

((datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.97')), (datetime.datetime(2019, 5, 29, 14, 54, 50), Decimal('34577.9

In [108]:
d = pymysql.connect(db='aiops_monitor', user='root', passwd='1234', host='58.16.78.136', port=23306)

sql = "select * from predict where predict predict_time='%s' and item_id='%d'" % ('2019-05-29 11:55:50','840')
cursor = d.cursor()
cursor.execute(sql)
d.commit()
d.close()

TypeError: %d format: a number is required, not str

In [2]:
import time
# 转换成localtime
time_local = time.localtime(1559115890)
# 转换成新的时间格式(2016-05-05 20:28:54)
dt = time.strftime("%Y-%m-%d %H:%M:%S", time_local)
dt

'2019-05-29 15:44:50'

In [5]:
import numpy as np
a=np.array([1,2,3])
b=np.array([2,3,4])
dist = np.linalg.norm(a-b)
dist

1.7320508075688772

In [6]:
np.max(a)

3

ModuleNotFoundError: No module named 'joblib'